# Análise de sentimento 

## Objetivo:

Elaborar um serviço que seja capaz de detectar se uma determinada frase inserida corresponde a um destes sentimentos (felicidade, tristeza e raiva)

Exemplo:

`Não quero saber de nada, eu te odeio!` —> `raiva`

`Sinto sua falta` —> `tristeza`

## Bibliotecas

In [15]:
# manipulação de dados
import pandas as pd

# manipulação de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#nltk.download('stopwords', quiet=True)
#stopwords_pt = set(stopwords.words('portuguese'))


## Base de dados:

Não encontrei uma base de dados classificando exatamente o sentimento dos textos, apenas classificando-os em **Positivos, Negativos ou Neutros**.

Por isto pedi ajuda ao ChatGPT para contruição da base para treinar o modelo.
A base consiste em apenas duas colunas: 

**Frase:** Frase aleatória gerada pelo ChatGPT;

**Sentimento**: Classificação se a Frase é de Felicidade, Tristeza ou Raiva.

In [3]:
frases_sentimentos = {
    "Estou muito feliz com este resultado!": "Felicidade", "Isso me deixa com muita raiva.": "Raiva",
    "Sinto-me triste com essa situação.": "Tristeza", "Estou tão animado para este evento!": "Felicidade",
    "Essa história me deixa arrepiado.": "Tristeza", "Fiquei tão furioso quando ouvi isso.": "Raiva",
    "Estou deprimido com as notícias de hoje.": "Tristeza", "Estou radiante com essa notícia!": "Felicidade",
    "Essa injustiça me deixa furioso.": "Raiva", "Meu coração está doendo de tristeza.": "Tristeza",
    "Estou chocado com essa revelação!": "Raiva", "Estou transbordando de felicidade!": "Felicidade",
    "Estou me sentindo tão miserável.": "Tristeza",  "Estou radiante com essa vitória!": "Felicidade",
    "Essa situação me deixa com tanto medo.": "Tristeza", "Essa injustiça me faz ferver de raiva.": "Raiva",
    "Me sinto completamente desamparado.": "Tristeza", "Estou eufórico com esta notícia!": "Felicidade",
    "Estou com muito medo do que pode acontecer depois.": "Tristeza", "Isso me faz perder o controle de tanta raiva.": "Raiva",
    "Estou de coração partido com essa notícia.": "Tristeza", "Estou transbordando de felicidade e emoção!": "Felicidade",
    "Essa situação me enfurece tanto.": "Raiva", "Estou devastado com essa tragédia.": "Tristeza",
    "Estou radiante com esta conquista!": "Felicidade", "Estou extasiado com essa descoberta!": "Felicidade",
    "Isso me deixa com uma alegria imensa.": "Felicidade", "Essa notícia me trouxe um sorriso enorme!": "Felicidade",
    "Fico tão feliz quando estou com meus amigos.": "Felicidade", "Estou radiante com o sucesso do projeto!": "Felicidade",
    "Estou tão triste por não poder ir ao evento.": "Tristeza", "Meu coração dói de saudade dos velhos tempos.": "Tristeza",
    "Não aguento mais essa sensação de vazio.": "Tristeza", "Sinto um aperto no peito ao lembrar da perda.": "Tristeza",
    "Estou devastado com essa notícia terrível.": "Tristeza", "Estou furioso com a incompetência dessa empresa.": "Raiva",
    "Essa atitude me deixa tão irritado!": "Raiva", "Não consigo suportar a falta de respeito.": "Raiva",
    "Estou com tanta raiva que mal consigo falar.": "Raiva","Essa injustiça me faz querer gritar de raiva.": "Raiva",
    "Estou transbordando de alegria com essa novidade!": "Felicidade", "Estou completamente apaixonado por esse filme!": "Felicidade",
    "Estou radiante com a chegada das férias!": "Felicidade","Que alegria poder passar tempo com a família!": "Felicidade",
    "Estou tão feliz que mal posso conter o sorriso.": "Felicidade","Estou triste por não poder estar presente na festa.": "Tristeza",
    "Essa lembrança traz lágrimas aos meus olhos.": "Tristeza","A solidão está me consumindo aos poucos.": "Tristeza",
    "Sinto um vazio imenso dentro de mim.": "Tristeza","Estou deprimido com a situação atual.": "Tristeza",
    "Estou revoltado com a falta de ética dessa empresa.": "Raiva", "Essa atitude é completamente inaceitável!": "Raiva",
    "Estou com raiva até o último fio de cabelo.": "Raiva", "Essa incompetência me deixa enfurecido!": "Raiva",
    "Estou tão zangado que mal consigo raciocinar direito.": "Raiva", "Estou radiante com a notícia do novo emprego!": "Felicidade",
    "Estou tão feliz que sinto borboletas no estômago!": "Felicidade", "Essa conquista é motivo de grande alegria!": "Felicidade",
    "Estou nas nuvens com o resultado do exame!": "Felicidade", "Que alegria poder realizar esse sonho!": "Felicidade",
    "Estou triste por não poder participar da celebração.": "Tristeza", "Essa perda é um golpe duro demais.": "Tristeza",
    "Sinto um peso no coração ao lembrar do passado.": "Tristeza", "A saudade é uma dor que não passa.": "Tristeza",
    "Estou tão triste que mal consigo me concentrar.": "Tristeza", "Estou revoltado com a corrupção neste país.": "Raiva",
    "Essa injustiça é um absurdo completo!": "Raiva", "Estou com uma raiva que não cabe em mim!": "Raiva",
    "Essa incompetência me deixa fora de mim!": "Raiva", "Estou tão enfurecido que mal consigo falar!": "Raiva",
    "Estou radiante com a notícia do casamento!": "Felicidade", "Estou tão feliz que mal consigo conter as lágrimas!": "Felicidade",
    "Que alegria poder compartilhar esse momento com vocês!": "Felicidade", "Estou tão feliz que sinto que vou explodir de alegria!": "Felicidade",
    "Que felicidade poder ver meu filho crescer!": "Felicidade",
    "Estou muito feliz com esta nova conquista!": "Felicidade", "Isso me deixa com uma alegria indescritível.": "Felicidade", "Essa notícia me trouxe um sorriso de orelha a orelha!": "Felicidade", "Fico tão feliz quando vejo o sucesso dos meus amigos.": "Felicidade", "Estou radiante com o resultado do projeto!": "Felicidade", "Estou tão triste por não poder estar presente no aniversário dela.": "Tristeza", "Meu coração dói de saudades dos velhos tempos.": "Tristeza", "Não suporto mais essa sensação de vazio que me consome.": "Tristeza", "Sinto um aperto no peito ao lembrar da perda que sofri.": "Tristeza", "Estou devastado com essa notícia terrível que recebi.": "Tristeza", "Estou furioso com a incompetência dessa empresa.": "Raiva", "Essa atitude me deixa tão irritado!": "Raiva", "Não tolero a falta de respeito.": "Raiva", "Estou com tanta raiva que mal consigo falar direito.": "Raiva", "Essa injustiça me faz querer gritar de raiva.": "Raiva", "Estou transbordando de alegria com essa novidade que recebi!": "Felicidade", "Estou completamente apaixonado por esse filme que acabei de assistir!": "Felicidade", "Estou radiante com a chegada das férias tão esperadas!": "Felicidade", "Que alegria poder passar tempo com a família reunida!": "Felicidade", "Estou tão feliz que mal posso conter o sorriso no rosto.": "Felicidade", "Estou triste por não poder estar presente na festa de aniversário dela.": "Tristeza", "Essa lembrança traz lágrimas aos meus olhos.": "Tristeza", "A solidão está me consumindo aos poucos.": "Tristeza", "Sinto um vazio imenso dentro de mim desde que você partiu.": "Tristeza", "Estou deprimido com a situação atual do mundo.": "Tristeza", "Estou revoltado com a falta de ética dessa empresa.": "Raiva", "Essa atitude é completamente inaceitável e me deixa revoltado!": "Raiva", "Estou com raiva até o último fio de cabelo.": "Raiva", "Essa incompetência me deixa enfurecido!": "Raiva", "Estou tão zangado que mal consigo raciocinar direito.": "Raiva", "Estou radiante com a notícia do novo emprego que consegui!": "Felicidade", "Estou tão feliz que sinto borboletas no estômago!": "Felicidade", "Essa conquista é motivo de grande alegria para mim!": "Felicidade", "Estou nas nuvens com o resultado positivo do exame!": "Felicidade", "Que alegria poder realizar esse sonho que tanto almejei!": "Felicidade", "Estou triste por não poder participar da celebração da vitória de vocês.": "Tristeza", "Essa perda é um golpe duro demais para superar.": "Tristeza", "Sinto um peso no coração ao lembrar do passado que ficou para trás.": "Tristeza", "A saudade é uma dor que não passa e me consome dia após dia.": "Tristeza", "Estou tão triste que mal consigo me concentrar nas atividades do dia a dia.": "Tristeza", "Estou revoltado com a corrupção que assola este país.": "Raiva", "Essa injustiça é um absurdo completo e me deixa revoltado!": "Raiva", "Estou com uma raiva que não cabe em mim!": "Raiva", "Essa incompetência me deixa fora de mim de tanta raiva!": "Raiva", "Estou tão enfurecido que mal consigo falar!": "Raiva", "Estou radiante com a notícia do casamento de vocês!": "Felicidade", "Estou tão feliz que mal consigo conter as lágrimas de emoção!": "Felicidade", "Que felicidade poder compartilhar esse momento tão especial com vocês!": "Felicidade", "Estou tão feliz que sinto que vou explodir de alegria a qualquer momento!": "Felicidade", "Que felicidade poder ver meu filho crescer saudável e feliz!": "Felicidade"
}



In [2]:
frases_sentimentos_adicionais = {"Estou muito feliz com esta nova conquista!": "Felicidade", "Isso me deixa com uma alegria indescritível.": "Felicidade", "Essa notícia me trouxe um sorriso de orelha a orelha!": "Felicidade", "Fico tão feliz quando vejo o sucesso dos meus amigos.": "Felicidade", "Estou radiante com o resultado do projeto!": "Felicidade", "Estou tão triste por não poder estar presente no aniversário dela.": "Tristeza", "Meu coração dói de saudades dos velhos tempos.": "Tristeza", "Não suporto mais essa sensação de vazio que me consome.": "Tristeza", "Sinto um aperto no peito ao lembrar da perda que sofri.": "Tristeza", "Estou devastado com essa notícia terrível que recebi.": "Tristeza", "Estou furioso com a incompetência dessa empresa.": "Raiva", "Essa atitude me deixa tão irritado!": "Raiva", "Não tolero a falta de respeito.": "Raiva", "Estou com tanta raiva que mal consigo falar direito.": "Raiva", "Essa injustiça me faz querer gritar de raiva.": "Raiva", "Estou transbordando de alegria com essa novidade que recebi!": "Felicidade", "Estou completamente apaixonado por esse filme que acabei de assistir!": "Felicidade", "Estou radiante com a chegada das férias tão esperadas!": "Felicidade", "Que alegria poder passar tempo com a família reunida!": "Felicidade", "Estou tão feliz que mal posso conter o sorriso no rosto.": "Felicidade", "Estou triste por não poder estar presente na festa de aniversário dela.": "Tristeza", "Essa lembrança traz lágrimas aos meus olhos.": "Tristeza", "A solidão está me consumindo aos poucos.": "Tristeza", "Sinto um vazio imenso dentro de mim desde que você partiu.": "Tristeza", "Estou deprimido com a situação atual do mundo.": "Tristeza", "Estou revoltado com a falta de ética dessa empresa.": "Raiva", "Essa atitude é completamente inaceitável e me deixa revoltado!": "Raiva", "Estou com raiva até o último fio de cabelo.": "Raiva", "Essa incompetência me deixa enfurecido!": "Raiva", "Estou tão zangado que mal consigo raciocinar direito.": "Raiva", "Estou radiante com a notícia do novo emprego que consegui!": "Felicidade", "Estou tão feliz que sinto borboletas no estômago!": "Felicidade", "Essa conquista é motivo de grande alegria para mim!": "Felicidade", "Estou nas nuvens com o resultado positivo do exame!": "Felicidade", "Que alegria poder realizar esse sonho que tanto almejei!": "Felicidade", "Estou triste por não poder participar da celebração da vitória de vocês.": "Tristeza", "Essa perda é um golpe duro demais para superar.": "Tristeza", "Sinto um peso no coração ao lembrar do passado que ficou para trás.": "Tristeza", "A saudade é uma dor que não passa e me consome dia após dia.": "Tristeza", "Estou tão triste que mal consigo me concentrar nas atividades do dia a dia.": "Tristeza", "Estou revoltado com a corrupção que assola este país.": "Raiva", "Essa injustiça é um absurdo completo e me deixa revoltado!": "Raiva", "Estou com uma raiva que não cabe em mim!": "Raiva", "Essa incompetência me deixa fora de mim de tanta raiva!": "Raiva", "Estou tão enfurecido que mal consigo falar!": "Raiva", "Estou radiante com a notícia do casamento de vocês!": "Felicidade", "Estou tão feliz que mal consigo conter as lágrimas de emoção!": "Felicidade", "Que felicidade poder compartilhar esse momento tão especial com vocês!": "Felicidade", "Estou tão feliz que sinto que vou explodir de alegria a qualquer momento!": "Felicidade", "Que felicidade poder ver meu filho crescer saudável e feliz!": "Felicidade"}


In [39]:
df = pd.DataFrame(frases_sentimentos.items(), columns=['Frase', 'Sentimento'])
df.head()

,Frase,Sentimento
0,Estou muito feliz com este resultado!,Felicidade
1,Isso me deixa com muita raiva.,Raiva
2,Sinto-me triste com essa situação.,Tristeza
3,Estou tão animado para este evento!,Felicidade
4,Essa história me deixa arrepiado.,Tristeza


In [51]:
df.shape

(75, 2)

## Preparação dos textos


### Stopwords

Pelo pacote nltk

In [12]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Ben-
[nltk_data]     Hur\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [14]:
lista_stopwords = nltk.corpus.stopwords.words('portuguese')
lista_stopwords

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

### Removendo as stopwords

In [42]:
df_stop = df.copy()

In [44]:
def remove_stopwords_from_text(text):
    tokens = word_tokenize(text, language='portuguese')
    filtered_tokens = [word for word in tokens if word.lower() not in lista_stopwords]
    return ' '.join(filtered_tokens)

# Remove as stopwords da coluna 'Frase'
df_stop['Frase Limpa'] = df_stop['Frase'].apply(remove_stopwords_from_text)
df_stop


,Frase,Sentimento,Frase Limpa
0,feliz resultado !,Felicidade,feliz resultado !
1,deixa muita raiva .,Raiva,deixa muita raiva .
2,Sinto-me triste situação .,Tristeza,Sinto-me triste situação .
3,tão animado evento !,Felicidade,tão animado evento !
4,história deixa arrepiado .,Tristeza,história deixa arrepiado .
...,...,...,...
70,radiante notícia casamento !,Felicidade,radiante notícia casamento !
71,tão feliz mal consigo conter lágrimas !,Felicidade,tão feliz mal consigo conter lágrimas !
72,alegria poder compartilhar momento !,Felicidade,alegria poder compartilhar momento !
73,tão feliz sinto vou explodir alegria !,Felicidade,tão feliz sinto vou explodir alegria !


### Representação de texto (Bag of word)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
countvec = CountVectorizer(ngram_range = (1,2))
countvec_data = countvec.fit_transform(df_stop['Frase Limpa'])

In [49]:
# Vocabulário criado a partir dos textos
print(countvec.vocabulary_)

{'feliz': 123, 'resultado': 254, 'feliz resultado': 126, 'deixa': 63, 'muita': 186, 'raiva': 247, 'deixa muita': 69, 'muita raiva': 187, 'sinto': 267, 'me': 177, 'triste': 302, 'situação': 275, 'sinto me': 271, 'me triste': 178, 'triste situação': 305, 'tão': 309, 'animado': 10, 'evento': 105, 'tão animado': 310, 'animado evento': 11, 'história': 150, 'arrepiado': 16, 'história deixa': 151, 'deixa arrepiado': 65, 'fiquei': 138, 'furioso': 140, 'ouvi': 203, 'fiquei tão': 139, 'tão furioso': 313, 'furioso ouvi': 142, 'deprimido': 75, 'notícias': 195, 'hoje': 152, 'deprimido notícias': 76, 'notícias hoje': 196, 'radiante': 241, 'notícia': 190, 'radiante notícia': 244, 'injustiça': 160, 'injustiça deixa': 162, 'deixa furioso': 67, 'coração': 55, 'doendo': 86, 'tristeza': 306, 'coração doendo': 56, 'doendo tristeza': 87, 'chocado': 30, 'revelação': 256, 'chocado revelação': 31, 'transbordando': 297, 'felicidade': 120, 'transbordando felicidade': 299, 'sentindo': 265, 'miserável': 182, 'sent

In [50]:
countvec_data.shape

(75, 333)

In [54]:
# Conversão da matriz esparsa para DataFrame
df_dtm = pd.DataFrame(countvec_data.toarray(), 
                      columns=countvec.get_feature_names())
df_dtm.index = df_stop.index
df_dtm = df_dtm.join(df_stop[['Sentimento']], how='left')
df_dtm

C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absurdo,absurdo completo,acontecer,aguento,aguento sensação,alegria,alegria imensa,alegria novidade,alegria poder,amigos,...,vitória,vou,vou explodir,zangado,zangado mal,ética,ética dessa,último,último fio,Sentimento
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Felicidade
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Raiva
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Tristeza
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Felicidade
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Tristeza
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Felicidade
71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Felicidade
72,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Felicidade
73,0,0,0,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,Felicidade
